In [ ]:
# Hay que depurar la lista de acciones para que solo aparezcan las que cotizan en las fechas seleccionadas.
# Por ejemplo, solo incluir las empresas que llevan cotizando desde el año x (hasta hoy)

In [ ]:
# SECCION 1: Inicializacion y descarga de datos 

from functions_bt import test, opt, index_epoch
import pandas as pd
import yfinance as yf
import numpy as np

# Devuelve un df con todos los tickers 
N = 200
xls = r'C:\Users\Usuario\Desktop\TFG\S&P 500 Companies (Standard and Poor 500).xlsx'
tickers = list(pd.read_excel(xls)['Symbol'][0:N])

# Inicializamos el periodo de analisis
start = '2000-01-01'; end = '2023-01-01'
timeperiod = pd.date_range(start = start, end = end, freq='B')
T = len(timeperiod)

# Inicializamos curva de equity
equity = pd.DataFrame({'Capital':0, 'Retorno Acumulado':0}, index = timeperiod)

# Descarga de los datos 
df = yf.download(tickers,start = start, end = end, show_errors=False)
df.dropna(axis=1, inplace=True)
tickers_d = df['Close'].columns.tolist()
print(f'Se han descartado {N-len(tickers_d)} de {N} tickers')

In [ ]:
# SECCION 2: PROCESO ITERATIVO: 
# NOTA: solo se esta teniendo en cuenta una variable a optimizar

# Inicializar las variables del bt:

epochs = 2                                  # Numero de epocas, cada epoca contiene optimizacion y test 
ratio = 1/3                                 # Proporcion periodos de optimizacion y test (optimizacion > test)
cons = list(np.arange(0.01,0.25,0.01))      # Restricciones de la optimizacion (de nuevo, solo una variable)

# Inicializar bases de datos: hiperparametros del sistema y parametros del backtesting
best_params_df = pd.DataFrame(index = tickers_d, columns = [f'Epoch{a}' for a in range(epochs)])
returns_df = pd.DataFrame(index = tickers_d, columns = [f'Epoch{a}' for a in range(epochs)])
vol_df = pd.DataFrame(index = tickers_d, columns = [f'Epoch{a}' for a in range(epochs)])
trades_df = pd.DataFrame(index = tickers_d, columns = [f'Epoch{a}' for a in range(epochs)])

for i in range(len(epochs)-1):
    
    # Indexa los datos correspondientes a cada epoca para el test y la optimizacion
    test_df, opt_df = index_epoch(df, epochs, ratio, i)

    # **** Inicio del proceso de optimizacion ****
    for j in range(len(tickers_d)):

        # Indexa la accion correspondiente para el periodo de optimizacion
        opt_stock = opt_df.xs(tickers_d[j], axis=1, level=1)

        # Optimiza y obtiene los mejores parametros de optimizacion
        best_params_df.iloc[j,i] = opt(opt_stock, strategy, cons, method = 'grid', goal = 'Return (Ann.) [%]', com = 0)

    # **** Inicio del test ****
    for k in range(tickers_d):
        test_stock = test_df.xs(tickers_d[j], axis=1, level=1)
        equity, mu, sigma, ntrades = test(test_stock, strategy, com = 0) 
        returns_df.iloc[k,i] = mu; vol_df.iloc[k,i] = sigma; trades_df.iloc[k,i] = ntrades
            
        # Aqui hay que armonizar la curva de equity y almacenar los parámetros del test  

    # Almacenar los datos del test: hiperparametros, curva de equity, resultados... de toda la muestra

In [ ]:
# SECCION 3: DISPLAY DE LOS RESULTADOS

# Plot de la curva de equity separada en epocas 
# Tabla comparativa con los mejores hiperparametros de cada epoca y resultados del sistema
